In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a Prophet Model using Vertex AI Tabular Workflows

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/prophet_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/prophet_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabular_workflows/prophet_on_vertex_pipelines.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>


## Overview

In this tutorial, you take on the role of a store planner who must determine how much inventory they need to order for each of their products and stores for November 2019. You accomplish this by fitting a Prophet model to your historical sales data using Vertex AI Tabular Workflows.

Prophet is a forecasting model maintained by Meta. It can handle both univariate and multivariate time series, as long as covariates are available when making a forecast. See the Prophet [paper](https://peerj.com/preprints/3190/) for algorithm details and the [documentation](https://facebook.github.io/prophet/) for more information about the library.

Like [BigQuery ML ARIMA_PLUS](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series), Prophet attempts to decompose each time series into trends, seasons, and holidays, producing a forecast using the aggregation of these models' predictions. One of many differences, however, is that BQML ARIMA+ uses ARIMA to model the trend component, while Prophet attempts to fit a curve using a piecewise logistic or linear model.

Google Cloud offers a pipeline for Prophet model training and another for Prophet batch prediction. Both pipelines are instances of [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) from [Google Cloud Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction) (GCPC).

The model training pipeline offers support for multiple time series. Because each Prophet model is designed for a single time series, the pipeline uses a [Vertex AI Custom Training Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) and [Dataflow](https://cloud.google.com/dataflow) to train multiple Prophet models in parallel. The model training pipeline performs hyperparameter tuning using [grid search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Grid_search) and Prophet's built-in backtesting logic.

Integration of Prophet with Vertex AI means that you can:

  * Use Vertex AI [data splitting](https://cloud.google.com/vertex-ai/docs/tabular-data/data-splits#forecasting) and [windowing strategies](https://cloud.google.com/vertex-ai/docs/tabular-data/bp-tabular#context-window).
  * Read data from either BigQuery tables or CSVs stored in Cloud Storage. Vertex AI expects each row to have the same format as for [Vertex AI Forecasting](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting/prepare-data).

Although Prophet is a multivariate model, Vertex AI does not yet have support for external regressors, so it can only be used as a univariate model.

Learn more about [Google Cloud Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction) and [Prophet for tabular data](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting-prophet).

### Objective

In this tutorial, you learn how to create several Prophet models using a training Vertex AI Pipeline from Google Cloud Pipeline Components (GCPC), and then do a batch prediction using the corresponding prediction pipeline.

This tutorial uses the following Google Cloud ML services and resources:

- BigQuery
- Cloud Storage
- Vertex AI
- Dataflow

The steps performed are:

1. Train the Prophet models.
1. View the evaluation metrics.
1. Make a batch prediction with the Prophet models.


### Dataset

This tutorial uses a synthetic dataset where product sales are dependent on a variety of factors such as advertisements, holidays, and locations.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery
* Dataflow

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Dataflow
pricing](https://cloud.google.com/dataflow/pricing), and [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 

In [ ]:
! (pip3 install --upgrade --quiet \
    google-cloud-aiplatform==1.40.0 \
    google-cloud-bigquery[pandas]==3.17.1 \
    google-cloud-pipeline-components==2.9.0)

### Colab only: Uncomment the following cell to restart the kernel.


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the following APIs: Vertex AI API, Cloud Resource Manager API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}
DATA_REGION = "us"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create Vertex AI Model resource and use for prediction.

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Service Account
You use a service account to create Vertex AI Pipeline jobs.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines
Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step. You only need to run this step once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/dataflow.developer

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import json
import os
import urllib

from google.cloud import aiplatform, bigquery
from google_cloud_pipeline_components.v1.automl.forecasting import utils

## Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Define train and prediction data

### Location of BigQuery destination table.

#### Create two datasets, one for each model you train. To make things simpler, create the datasets in the same region as the training data.

In [ ]:
dataset_name = f"forecasting_demo_prophet_{UUID}"

dataset_path = ".".join([PROJECT_ID, dataset_name])

# Must be same region as TRAINING_DATASET_BQ_PATH.
client = bigquery.Client(project=PROJECT_ID)
bq_dataset = bigquery.Dataset(dataset_path)
bq_dataset.location = DATA_REGION
bq_dataset = client.create_dataset(bq_dataset)
print(f"Created bigquery dataset {dataset_path} in {DATA_REGION}")

### Location of BigQuery training data.

Before training a model, you must first generate our dataset of store sales. This dataset includes multiple products and stores, and it also simulates factors such as advertisements and holiday effects. The data is be split into `TRAIN`, `VALIDATE`, `TEST`, and `PREDICT` sets, where the last three sets are all 1 month in duration.

#### Begin by defining the subqueries that create this base sales data.

In [ ]:
base_data_query = """
  WITH

    -- Create time series for each product + store with some covariates.
    time_series AS (
      SELECT
        CONCAT("id_", store_id, "_", product_id) AS id,
        CONCAT('store_', store_id) AS store,
        CONCAT('product_', product_id) AS product,
        date,
        -- Advertise 1/100 products.
        IF(
          ABS(MOD(FARM_FINGERPRINT(CONCAT(product_id, date)), 100)) = 0,
          1,
          0
        ) AS advertisement,
        -- Mark Thanksgiving sales as holiday sales.
        IF(
          EXTRACT(DAYOFWEEK FROM date) = 6
            AND EXTRACT(MONTH FROM date) = 11
            AND EXTRACT(DAY FROM date) BETWEEN 23 AND 29,
          1,
          0
        ) AS holiday,
        -- Set when each data split ends.
        CASE
          WHEN date < '2019-09-01' THEN 'TRAIN'
          WHEN date < '2019-10-01' THEN 'VALIDATE'
          WHEN date < '2019-11-01' THEN 'TEST'
          ELSE 'PREDICT'
        END AS split,
      -- Generate the sales with one SKU per date.
      FROM
        UNNEST(GENERATE_DATE_ARRAY('2017-01-01', '2019-12-01')) AS date
      CROSS JOIN
        UNNEST(GENERATE_ARRAY(0, 10)) AS product_id
      CROSS JOIN
        UNNEST(GENERATE_ARRAY(0, 3)) AS store_id
    ),

    -- Randomly determine factors that contribute to how syntheic sales are calculated.
    time_series_sales_factors AS (
      SELECT
        *,
        ABS(MOD(FARM_FINGERPRINT(product), 10)) AS product_factor,
        ABS(MOD(FARM_FINGERPRINT(store), 10)) AS store_factor,
        [1.6, 0.6, 0.8, 1.0, 1.2, 1.8, 2.0][
          ORDINAL(EXTRACT(DAYOFWEEK FROM date))] AS day_of_week_factor,
        1 +  SIN(EXTRACT(MONTH FROM date) * 2.0 * 3.14 / 24.0) AS month_factor,
        -- Advertised products have increased sales factors for 5 days.
        CASE
          WHEN LAG(advertisement, 0) OVER w = 1.0 THEN 1.2
          WHEN LAG(advertisement, 1) OVER w = 1.0 THEN 1.8
          WHEN LAG(advertisement, 2) OVER w = 1.0 THEN 2.4
          WHEN LAG(advertisement, 3) OVER w = 1.0 THEN 3.0
          WHEN LAG(advertisement, 4) OVER w = 1.0 THEN 1.4
          ELSE 1.0
        END AS advertisement_factor,
        IF(holiday = 1.0, 2.0, 1.0) AS holiday_factor,
        0.001 * ABS(MOD(FARM_FINGERPRINT(CONCAT(product, store, date)), 100)) AS noise_factor
      FROM
        time_series
      WINDOW w AS (PARTITION BY id ORDER BY date)
    ),

    -- Use factors to calculate synthetic sales for each time series.
    base_data AS (
      SELECT
        id,
        store,
        product,
        date,
        split,
        advertisement,
        holiday,
        (
          (1 + store_factor)
          * (1 + product_factor)
          * (1 + month_factor + day_of_week_factor)
          * (
            1.0
            + 2.0 * advertisement_factor
            + 3.0 * holiday_factor
            + 5.0 * noise_factor
          )
        ) AS sales
      FROM
        time_series_sales_factors
      )
"""

Next, convert this base sales data into a dataset you use to train a model, and a dataset you pass to a trained model at serving time. The training dataset includes the `TRAIN`, `VALIDATE`, and `TEST` splits, while the prediction dataset includes the `PREDICT` split and also the `TEST` split to provide context information.

In [ ]:
TRAINING_DATASET_BQ_PATH = f"{dataset_path}.train"
PREDICTION_DATASET_BQ_PATH = f"{dataset_path}.pred"

train_query = f"""
    CREATE OR REPLACE TABLE `{dataset_path}.train` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split != 'PREDICT'
"""
client.query(train_query).result()
print(f"Created {TRAINING_DATASET_BQ_PATH}.")

pred_query = f"""
    CREATE OR REPLACE TABLE `{dataset_path}.pred` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split = 'TEST'

    UNION ALL

    SELECT * EXCEPT (sales), NULL AS sales
    FROM base_data
    WHERE split = 'PREDICT'
"""
client.query(pred_query).result()
print(f"Created {PREDICTION_DATASET_BQ_PATH}.")

You can take a look at the sales data that was generated. Later in this tutorial, we visualize the time series along with our forecast.

The model is trained with data from January 2017 to October 2019 inclusive.

#### Look at the training data

In [ ]:
query = f"SELECT * FROM `{dataset_path}.train` LIMIT 10"
client.query(query).to_dataframe().head()

The table used for prediction contains data from November 2019. It also includes actuals from October 2019 as context information.

#### Look at the prediction data

In [ ]:
query = f"SELECT * FROM `{dataset_path}.pred` LIMIT 10"
client.query(query).to_dataframe().head()

## Create a Prophet model

Use GCPC to get the Prophet pipeline to start an `aiplatform.PipelineJob`.

Because Prophet models can only fit a single time series, this training pipeline uses a [Vertex AI Custom Training Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) to train multiple Prophet models in parallel. The number of models trained equals the number of unique values in the `time_series_identifier_column` multiplied by the number of hyperparameter tuning trials determined by `max_num_trials`. [Dataflow](https://cloud.google.com/dataflow/docs/about-dataflow) is used to parallelize these model trainings.

For hyperparameter tuning, the Prophet training job performs a deterministic [grid search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Grid_search) over the [parameters](https://facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning) recommended by the Prophet documentation. The metric used for tuning is specified by the `optimization_objective` parameter, and is calculated using Prophet's `cross_validation` function over the validation split.

### Create the training job

Use the `get_prophet_train_pipeline_and_parameters` function to configure your training job.

-  `project` (str): The GCP project that runs the pipeline components.
-  `location` (str): The GCP region for Vertex AI.
-  `root_dir` (str): The Cloud Storage location to store the output.
-  `time_column` (str): Name of the column that identifies time order in the time series.
-  `time_series_identifier_column` (str): Name of the column that identifies the time series.
-  `target_column` (str): Name of the column that the model is to predict values for.
-  `forecast_horizon` (int): The number of time periods into the future for which forecasts are created. Future periods start after the latest timestamp for each time series.
-  `optimization_objective` (str): Optimization objective for tuning. Supported metrics come from Prophet's performance_metrics function. These are mse, rmse, mae, mape, mdape, smape, and coverage.
-  `data_granularity_unit` (str): String representing the units of time for the time column.
- `data_source_bigquery_table_path` (str): The BigQuery table path of format bq://bq_project.bq_dataset.bq_table
- `dataflow_service_account` (str): Custom service account to run dataflow jobs. The dataflow job can also be configured to use private IPs and a specific VPC subnet.
- `max_num_trials` (int): Maximum number of tuning trials to perform per time series.
-  `predefined_split_key` (str): The name of the predefined_split column. Vertex AI splits the data using the TRAIN, VALIDATE, or TEST values in this column. To learn more about data splits and for the full set of data split parameters, refer to the public documentation page [Forecasting with Prophet](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting-prophet).
-  `window_stride_length` (int): Step length used to generate input examples. Every window_stride_length rows is used to generate a sliding window. To learn more about windowing and for the full set of windowing parameters, refer to the public documentation page [Forecasting with Prophet](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting-prophet).
-  `trainer_dataflow_machine_type` (str): The Dataflow machine type used for training.
- `trainer_dataflow_max_num_workers` (int): The maximum number of Dataflow workers used for training.
- `evaluation_dataflow_machine_type` (int): The Dataflow machine type used for evaluation.
- `evaluation_dataflow_max_num_workers` (int): The max number of Dataflow workers used for evaluation.

Prophet training jobs run on Dataflow, meaning there is an initial startup time of 5-7 minutes. Any runtime past this startup time can be improved by scaling vertically (ex. changing the machine type from `n1-standard-2` to `e2-highcpu-8`) or horizontally (ex. increasing the number of workers from `5` to `100`).

For a full list of parameters, see the GCPC SDK [documentation](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.9.0/api/v1/automl/forecasting.html#v1.automl.forecasting.get_prophet_train_pipeline_and_parameters).


In [ ]:
time_column = "date"  # @param {type: "string"}
time_series_identifier_column = "id"  # @param {type: "string"}
target_column = "sales"  # @param {type: "string"}
forecast_horizon = 30  # @param {type: "integer"}
optimization_objective = "rmse"  # @param {type: "string"}
data_granularity_unit = "day"  # @param {type: "string"}
split_column = "split"  # @param {type: "string"}
window_stride_length = 1  # @param {type: "integer"}

(
    train_job_spec_path,
    train_parameter_values,
) = utils.get_prophet_train_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    root_dir=os.path.join(BUCKET_URI, "pipeline_root"),
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    target_column=target_column,
    forecast_horizon=forecast_horizon,
    optimization_objective=optimization_objective,
    data_granularity_unit=data_granularity_unit,
    predefined_split_key=split_column,
    data_source_bigquery_table_path=TRAINING_DATASET_BQ_PATH,
    window_stride_length=window_stride_length,
    max_num_trials=2,
    trainer_dataflow_machine_type="n1-standard-2",
    trainer_dataflow_max_num_workers=5,
    evaluation_dataflow_machine_type="n1-standard-1",
    evaluation_dataflow_max_num_workers=1,
    dataflow_service_account=SERVICE_ACCOUNT,
)

### Run the training pipeline

Use the Vertex AI Python SDK to kick off a training pipeline run. Once the run has started, the following cell outputs a link that allows you to monitor the run. The link should look like this:

`https://console.cloud.google.com/vertex-ai/locations/[REGION]/pipelines/runs/[DISPLAY_NAME]`

In [ ]:
# The display name should be unique even if this cell is rerun.
DISPLAY_NAME = f"forecasting-demo-train-{generate_uuid()}"

job = aiplatform.PipelineJob(
    job_id=DISPLAY_NAME,
    display_name=DISPLAY_NAME,
    pipeline_root=os.path.join(BUCKET_URI, DISPLAY_NAME),
    template_path=train_job_spec_path,
    parameter_values=train_parameter_values,
    enable_caching=False,
)
job.run(service_account=SERVICE_ACCOUNT)

If you ever want to reuse an existing run, the above command can be replaced with:

```
job = aiplatform.PipelineJob.get('projects/[PROJECT_NUMBER]/locations/[REGION]/pipelineJobs/[PIPELINE_RUN_NAME]')

```

## Review model evaluation scores
After your model has finished training, you can review the evaluation scores for it.

#### Metrics are always reported via the `metrics` table in the destination dataset.

In [ ]:
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "model-evaluation-regression":
        metrics = task_detail.outputs["evaluation_metrics"].artifacts[0].metadata
        break
else:
    raise ValueError("Couldn't find the model evaluation task.")

print("Evaluation metrics:\n")
dict(metrics)

## Create and run prediction job

### Create prediction job

Like with the training job, get the prediction pipeline from GCPC and use it to create an `aiplatform.PipelineJob`.

Because there is one Prophet model per time series ID, the Prophet prediction server expects inputs to be aggregated by time series ID and outputs predictions using the same aggregation. This prediction pipeline automates the process of aggregating inputs and disaggregating the outputs from batch prediction.

The prediction job expects the following parameters:
-  `project` (str): The GCP project that runs the pipeline components.
-  `location` (str): The GCP region for Vertex AI.
-  `model_name` (str): The name of the Model resource, in a form of
      projects/{project}/locations/{location}/models/{model}.
-  `time_column` (str): Name of the column that identifies time order in the time series.
-  `time_series_identifier_column` (str): Name of the column that identifies the time series.
-  `target_column` (str): Name of the column that the model is to predict values for.
-  `data_source_csv_filenames` (str): A string that represents a list of comma separated CSV filenames.
-  `data_source_bigquery_table_path` (str): The BigQuery table path of format bq://bq_project.bq_dataset.bq_table
-  `bigquery_destination_uri` (str): URI of the desired destination dataset. If not specified, resources are created under a new dataset in the project.
-  `machine_type` (str): The machine type used for batch prediction.
-  `max_num_workers` (int): The maximum number of workers used for batch prediction.

For a full list of parameters, see the GCPC SDK [documentation](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.9.0/api/v1/automl/forecasting.html#v1.automl.forecasting.get_prophet_prediction_pipeline_and_parameters).

In [ ]:
# Get the model name programmatically, you can find this by looking at the
# execution graph in Vertex AI Pipelines.
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "model-upload":
        model = task_detail.outputs["model"].artifacts[0].metadata["resourceName"]
        break
else:
    raise ValueError("Couldn't find the model training task.")

# Use the model when creating the pipeline parameters.
(
    prediction_job_spec_path,
    prediction_parameter_values,
) = utils.get_prophet_prediction_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    model_name=model,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    target_column=target_column,
    data_source_bigquery_table_path=PREDICTION_DATASET_BQ_PATH,
    bigquery_destination_uri=dataset_path,
    machine_type="n1-standard-2",
    max_num_workers=5,
)

### Run the prediction pipeline

Use the Vertex AI Python SDK to kick off a prediction pipeline run. Once the run has started, the following cell outputs a link that allows you to monitor the run. The link should look like this:

`https://console.cloud.google.com/vertex-ai/locations/[REGION]/pipelines/runs/[DISPLAY_NAME]`

In [ ]:
# The display name should be unique even if this cell is rerun.
DISPLAY_NAME = f"forecasting-demo-predict-{generate_uuid()}"

job = aiplatform.PipelineJob(
    job_id=DISPLAY_NAME,
    display_name=DISPLAY_NAME,
    pipeline_root=os.path.join(BUCKET_URI, DISPLAY_NAME),
    template_path=prediction_job_spec_path,
    parameter_values=prediction_parameter_values,
    enable_caching=False,
)
job.run(service_account=SERVICE_ACCOUNT)

### Get the predictions

Next, get the results from the completed batch prediction job. These are always written to a table called `predictions` under the output dataset.

In [ ]:
# Get the prediction table programmatically, you can find this by looking at the
# execution graph in Vertex AI Pipelines.
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "bigquery-query-job-2":
        pred_table = (
            task_detail.outputs["destination_table"].artifacts[0].metadata["tableId"]
        )
        break
else:
    raise ValueError("Couldn't find the prediction task.")

query = f"SELECT * FROM `{dataset_path}.{pred_table}`"
preds = client.query(query).to_dataframe()
preds.head()

## Visualize the forecasts

Lastly, follow the given link to visualize the generated forecasts in [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en).
The code block included in this section dynamically generates a Data Studio link that specifies the template, the location of the forecasts, and the query to generate the chart. The data is populated from the forecasts generated earlier.

You can inspect the used template at https://datastudio.google.com/c/u/0/reporting/067f70d2-8cd6-4a4c-a099-292acd1053e8. This was created by Google specifically to view forecasting predictions.

In [ ]:
def _sanitize_bq_uri(bq_uri: str):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri: str,
    batch_prediction_bq_output_uri: str,
    time_column: str,
    time_series_identifier_column: str,
    target_column: str,
):
    """Creates a link that fills in the demo Data Studio template."""
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    query = f"""
        SELECT
          CAST(input.{time_column} as DATETIME) timestamp_col,
          CAST(input.{time_series_identifier_column} as STRING) time_series_identifier_col,
          CAST(input.{target_column} as NUMERIC) historical_values,
          CAST(predicted_{target_column}.value as NUMERIC) predicted_values,
        FROM `{batch_prediction_bq_input_uri}` input
        LEFT JOIN `{batch_prediction_bq_output_uri}` output
          ON
            TIMESTAMP(input.{time_column}) = TIMESTAMP(output.{time_column})
            AND CAST(input.{time_series_identifier_column} as STRING) = CAST(
              output.{time_series_identifier_column} as STRING)
    """
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    url_params = urllib.parse.urlencode({"params": json.dumps(params)})
    return f"{base_url}?{url_params}"

In [ ]:
actuals_table = f"{dataset_path}.actuals"
query = f"""
    CREATE OR REPLACE TABLE `{actuals_table}` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split != 'TRAIN'
"""
client.query(query).result()
print(f"Created {actuals_table}.")

In [ ]:
print("Click the link below to view Prophet predictions:")
print(
    get_data_studio_link(
        batch_prediction_bq_input_uri=actuals_table,
        batch_prediction_bq_output_uri=f"{dataset_path}.{pred_table}",
        time_column=time_column,
        time_series_identifier_column=time_series_identifier_column,
        target_column=target_column,
    )
)

## Clean up Vertex AI and BigQuery resources

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Model
- Cloud Storage Bucket
- BigQuery tables

In [ ]:
# Delete the model
aiplatform.Model(model).delete()

# Delete output datasets
client.delete_dataset(dataset_path, delete_contents=True, not_found_ok=True)

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI